# Mamadou Mourtalla NDjitte 
L3 TDSI , Projet 2
mourtalladjite@outlook.fr

# Partie 2 : Implémentation de DES et AES

## Utile:

In [1]:
import random

def decalage_gauche(m,i):
    n=''
    if i==1:
        while i < len(m):
            n +=m[i]
            i+=1
        n += m[0]
    else :
        while i < len(m):
            n += m[i]
            i += 1
        n +=m[0]
        n +=m[1]
    return n

dico = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,
        'J':9,'K':10,'L':11,'M':12,'N':13,'O':14,'P':15,'Q':16,'R':17
       ,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,'Z':25}

def format_message(message):
    message = message.upper()
    message_g=''
    e=["Ê","Ë","É","È"] #è,é,..org(é)
    for i in message:
        if i in e :
            i=i.replace(i,'E')
        if ( i in dico.keys() ):
            message_g += i
    return message_g

# Convertir du texte ASCII en binaire
def ascii_to_bin(M):
    M = format_message(M)
    b = ''.join(format(ord(x), 'b') for x in M) #org recupere le num ASCII ex 'A'=65 on le trensforme en binaire avec format 
    return str(b)
    
    

In [2]:
ascii_to_bin("Hello word")

'100100010001011001100100110010011111010111100111110100101000100'

# DES

Le Data Encryption Standard (DES, prononcer /dɛs/) est un algorithme de chiffrement symétrique (chiffrement par bloc) utilisant des clés de 56 bits.

D'une manière générale, on peut dire que DES fonctionne en trois étapes :

- permutation initiale et fixe d'un bloc (sans aucune incidence sur le niveau de sécurité) ;
- le résultat est soumis à 16 itérations d'une transformation, ces itérations dépendent à chaque tour d'une autre clé partielle de 48 bits. Cette clé de tour intermédiaire est calculée à partir de la clé initiale de l'utilisateur (grâce à un réseau de tables de substitution et d'opérateurs XOR). Lors de chaque tour, le bloc de 64 bits est découpé en deux blocs de 32 bits, et ces blocs sont échangés l'un avec l'autre selon un schéma de Feistel. Le bloc de 32 bits ayant le poids le plus fort (celui qui s'étend du bit 32 au bit 64) subira une transformation ;
- le résultat du dernier tour est transformé par la fonction inverse de la permutation initiale.

**Mécanisme :**

Le message, au prélable converti en binaire, est découpé en blocs Bi de 64 bits. La clé K, elle, comporte 56 bits. Pour chaque bloc Bi, on applique l'algorithme* suivant :

- On effectue une permutation initiale des bits du bloc Bi. On appelle alors G0 et D0 les parties de 32 bits droite et gauche du bloc obtenu.

- On répète 16 fois la procédure suivante :
     Gi = Di-1
     Di = Gi-1 XOR f(Di-1 , Ki)    (XOR est représenté par + sur le schéma ci-dessous)
     où Ki est un bloc de 48 bits de la clé K, et f une fonction composée successivement d'une expansion de bits, d'un XOR, d'une réduction de bits, et d'une permutation de bits.
- On recompose un bloc B'16 en "recollant" D16 et G16 dans cet ordre.
- On effectue la permutation inverse de la permutation initiale 1).

Aujourd'hui, le D.E.S. est fortement menacé par les puissances de calcul des ordinateurs. Il n'est en effet pas impossible de balayer la plupart des clés pour casser le code. 

https://www.commentcamarche.net/contents/204-introduction-au-chiffrement-avec-des

http://page.math.tu-berlin.de/~kant/teaching/hess/krypto-ws2006/des.htm

### 1_Genkey_DES

 La clé initiale K est sur 64 bits.
- La permutation PC-1 enlève les bits de parité et opère sur les 56 bits restants.
- On divise le résultat en deux moitiés C0et D0 de 28 bits.
  
- On génère une suite Ci, Di en opérant des décalages à gauche successifs :
 
Ci = LSi (Ci-1)
Di = LSi (Di-1)
  
- Pour obtenir la clé Ki on regroupe Ci et Di et l'on opère sur les 56 bits une permutation PC-2
 
Ki = PC-2(Ci Di)

La clé 64 bits est permutée selon le tableau suivant, PC-1. Depuis la première entrée dans le tableau est de 57 , cela signifie que le 57ème bit de la clé d’origine K devient le premier bit de la clé permutée K . Le 49ème bit de la touche originale devient le deuxième bit de la clé permutée. Le 4ème bit de la clé d’origine est le dernier morceau de la clé permutée. Notez que 56 bits de la clé d’origine apparaissent dans la clé permutée.

PC-1

In [3]:
def genkey_system_des():
    b = bin(random.getrandbits(64)) #Renvoie un entier de 64 bits
    k = b[2:].zfill(64) #enleve le '0b' et ajoute un 0 au debut pour avoir 64 bit (0b101 va devenir 0101)
    k = str(k)
    k = '0001001100110100010101110111100110011011101111001101111111110001' #test
    #print(f"Clef de 64 bits: \n k = {k}")
    k_pc1 = ''
    pc1=[ 57, 49, 41, 33, 25, 17 ,9,
           1, 58, 50, 42, 34, 26, 18,
          10, 2, 59, 51, 43, 35, 27,
          19, 11, 3, 60, 52, 44, 36,
          63, 55, 47, 39, 31, 23, 15,
           7, 62, 54, 46, 38, 30, 22,
          14, 6, 61, 53, 45, 37, 29,
          21, 13, 5, 28, 20, 12, 4
        ]
    #permutation d k suivant pc-1
    for i in pc1:
        k_pc1 += k[i-1]
    #print(f"Permutation de k suivant pc-1 : \n k_pc-1 = {k_pc1}")
    return k_pc1

In [9]:
genkey_system_des()

'11110000110011001010101011110101010101100110011110001111'

In [18]:
def genkey_subkeys_system_des( k_pc1=genkey_system_des()):
    #determination de C0 et D0
    co =k_pc1[:28]
    do =k_pc1[28:]
    #print(f"Co=  {co} et Do = {do} ")
    print('determination de Cn et Dn avec 1<=n<=16')
    left_shift = [1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,1]
    dict_keyc = {0:co}
    dict_keyd = {0:do}
    c , d ,j = co ,do , 1
    for i in left_shift:
        dict_keyc[j] = decalage_gauche(c,i)
        dict_keyd[j] = decalage_gauche(d,i)
        c , d = dict_keyc[j], dict_keyd[j]
        j += 1 
    for i in range(17):
        #print(f"c{i}  : {dict_keyc[i]}  , d{i}  : {dict_keyd[i]}")
        pass
    pc2=[14 ,17 ,11 ,24 ,1 ,5
         ,3 ,28 ,15 ,6 ,21 ,10
         ,23 ,19 ,12 ,4 ,26 ,8
         ,16 ,7 ,27 ,20 ,13, 2
         ,41 ,52 ,31 ,37 ,47 ,55
         ,30 ,40 ,51 ,45 ,33 ,48
         ,44 ,49 ,39 ,56 ,34 ,53
         ,46 ,42 ,50 ,36 ,29 ,32
        ]
    s,sp='',''
    keys = []
    #print(f"Permutation de chacune des paires concaténées Cn Dn  suivant pc-2 ")
    for i in range(1,17):
        s = dict_keyc[i] + dict_keyd[i]
        for j in pc2:
            sp += s[j-1]
        keys.append(sp)
        #print(f"k{i} = {sp}")
        sp=''
    return keys 

In [19]:
 genkey_subkeys_system_des()

determination de Cn et Dn avec 1<=n<=16


['000110110000001011101111111111000111000001110010',
 '011110011010111011011001110110111100100111100101',
 '010101011111110010001010010000101100111110011001',
 '011100101010110111010110110110110011010100011101',
 '011111001110110000000111111010110101001110101000',
 '011000111010010100111110010100000111101100101111',
 '111011001000010010110111111101100001100010111100',
 '111101111000101000111010110000010011101111111011',
 '111000001101101111101011111011011110011110000001',
 '101100011111001101000111101110100100011001001111',
 '001000010101111111010011110111101101001110000110',
 '011101010111000111110101100101000110011111101001',
 '100101111100010111010001111110101011101001000001',
 '010111110100001110110111111100101110011100111010',
 '101111111001000110001101001111010011111100001010',
 '110010110011110110001011000011100001011111110101']

### 2_Cipher_DES

ci, le 58e bit de M est "1", qui devient le premier bit d' IP . Le 50e bit de M est "1", qui devient le deuxième bit d' IP . Le 7e bit de M est "0", qui devient le dernier bit d' IP .

Divisez ensuite le bloc permuté IP en une moitié gauche L 0 de 32 bits et une moitié droite R 0 de 32 bits.

On procède maintenant à 16 itérations, pour 1 <= n <= 16, en utilisant une fonction f qui opère sur deux blocs - un bloc de données de 32 bits et une clé K n de 48 bits - pour produire un bloc de 32 bits . Soit + dénote l'addition XOR, (addition bit à bit modulo 2) . Ensuite, pour n allant de 1 à 16, nous calculons

L n = R n-1

R n = L n-1 + f ( R n-1 , K n )

Il en résulte un bloc final, pour n = 16, de L 16 R 16 . Autrement dit, à chaque itération, nous prenons les 32 bits de droite du résultat précédent et en faisons les 32 bits de gauche de l'étape en cours. Pour les 32 bits de droite de l'étape en cours, on XOR les 32 bits de gauche de l'étape précédente avec le calcul f .


In [76]:
def cipher_system_des(M , k = genkey_subkeys_system_des()):
    m = format_message(M)
    mb = ascii_to_bin(m)
    
    #test
    mb = '0000000100100011010001010110011110001001101010111100110111101111' 
    
    ip=[58 ,50 ,42 ,34 ,26 ,18 ,10 ,2
        ,60 ,52 ,44 ,36 ,28 ,20 ,12 ,4
        ,62 ,54 ,46 ,38 ,30 ,22 ,14 ,6
        ,64 ,56 ,48 ,40 ,32 ,24 ,16 ,8
        ,57 ,49 ,41 ,33 ,25 ,17 ,9 ,1
        ,59 ,51 ,43 ,35 ,27 ,19 ,11 ,3
        ,61 ,53 ,45 ,37 ,29 ,21 ,13 ,5
        ,63 ,55 ,47 ,39 ,31 ,23 ,15 ,7
       ]
   
    k_ip =''
    #permutation de m suivant ip
    for i in ip:
        k_ip += mb[i-1]
    print(f"Permutation de  {M} suivant ip : \n k_ip = {k_ip}") 
    
    #determination de Lo et Ro
    lo = k_ip[:32]
    ro = k_ip[32:]
    print('determination de Ln et Rn avec 1<=n<=16')   
    #transforme chaque bloc Rn-1 de 32 bits à 48 bits
    def extension(bit32):
        bit48=''
        tab_selc_bit=[32 ,1 ,2 ,3 ,4 ,5
                      ,4 ,5 ,6 ,7 ,8 ,9
                      ,8 ,9 ,10 ,11 ,12 ,13
                     ,12 ,13 ,14 ,15 ,16 ,17
                     ,16 ,17 ,18 ,19 ,20 ,21
                     ,20 ,21 ,22 ,23 ,24 ,25
                     ,24 ,25 ,26 ,27 ,28 ,29
                     ,28 ,29 ,30 ,31 ,32 ,1]
        for i in tab_selc_bit:
            bit48 += bit32[i-1]
        #print(f"ex {bit48}")
        return bit48
    
    #calcul f , on XOR la sortie E ( Rn-1 ) avec la clé K n :
    def xor(bit48 ,  k):
        x = ''
        for i in range (48) :
            if bit48[i] == k[i]:
                x += '0'
            else :
                x += '1'
        #print(f"xor {x}")       
        return x

    def bit_par_6(x):
        s,j = '',0
        B = []
        for i in range(48) :
            s += x[i]
            j +=1
            if j == 6:
                B.append(s)
                s,j = '',0
        #print(f"bit par 6 {B}")
        return B
    
    def Sbox(bit48_6b):  
        s = [
             # s1
             [[14,  4, 13,  1,  2, 15, 11,  8,  3, 10,  6, 12,  5,  9,  0,  7],
              [ 0, 15,  7,  4, 14,  2, 13,  1, 10,  6, 12, 11,  9,  5,  3,  8],
              [ 4,  1, 14,  8, 13,  6,  2, 11, 15, 12,  9,  7,  3, 10,  5,  0],
              [15, 12,  8,  2,  4,  9,  1,  7,  5, 11,  3, 14, 10,  0,  6, 13]],
             # s2
             [[15,  1,  8, 14,  6, 11,  3,  4,  9,  7,  2, 13, 12,  0,  5, 10],
              [ 3, 13,  4,  7, 15,  2,  8, 14, 12,  0,  1, 10,  6,  9, 11,  5],
              [ 0, 14,  7, 11, 10,  4, 13,  1,  5,  8, 12,  6,  9,  3,  2, 15],
              [13,  8, 10,  1,  3, 15,  4,  2, 11,  6,  7, 12,  0,  5, 14,  9]],
             # s3
             [[10,  0,  9, 14,  6,  3, 15,  5,  1, 13, 12,  7, 11,  4,  2,  8],
              [13,  7,  0,  9,  3,  4,  6, 10,  2,  8,  5, 14, 12, 11, 15,  1],
              [13,  6,  4,  9,  8, 15,  3,  0, 11,  1,  2, 12,  5, 10, 14,  7],
              [ 1, 10, 13,  0,  6,  9,  8,  7,  4, 15, 14,  3, 11,  5,  2, 12]],
             # s4
             [[ 7, 13, 14,  3,  0,  6,  9, 10,  1,  2,  8,  5, 11, 12,  4, 15],
              [13,  8, 11,  5,  6, 15,  0,  3,  4,  7,  2, 12,  1, 10, 14,  9],
              [10,  6,  9,  0, 12, 11,  7, 13, 15,  1,  3, 14,  5,  2,  8,  4],
              [ 3, 15,  0,  6, 10,  1, 13,  8,  9,  4,  5, 11, 12,  7,  2, 14]],
             # s5
             [[ 2, 12,  4,  1,  7, 10, 11,  6,  8,  5,  3, 15, 13,  0, 14,  9],
              [14, 11,  2, 12,  4,  7, 13,  1,  5,  0, 15, 10,  3,  9,  8,  6],
              [ 4,  2,  1, 11, 10, 13,  7,  8, 15,  9, 12,  5,  6,  3,  0, 14],
              [11,  8, 12,  7,  1, 14,  2, 13,  6, 15,  0,  9, 10,  4,  5,  3]],
             # s6
             [[12,  1, 10, 15,  9,  2,  6,  8,  0, 13,  3,  4, 14,  7,  5, 11],
              [10, 15,  4,  2,  7, 12,  9,  5,  6,  1, 13, 14,  0, 11,  3,  8],
              [ 9, 14, 15,  5,  2,  8, 12,  3,  7,  0,  4, 10,  1, 13, 11,  6],
              [ 4,  3,  2, 12,  9,  5, 15, 10, 11, 14,  1,  7,  6,  0,  8, 13]],
             # s7
             [[ 4, 11,  2, 14, 15,  0,  8, 13,  3, 12,  9,  7,  5, 10,  6,  1],
              [13,  0, 11,  7,  4,  9,  1, 10, 14,  3,  5, 12,  2, 15,  8,  6],
              [ 1,  4, 11, 13, 12,  3,  7, 14, 10, 15,  6,  8,  0,  5,  9,  2],
              [ 6, 11, 13,  8,  1,  4, 10,  7,  9,  5,  0, 15, 14,  2,  3, 12]],
             # s8
             [[13,  2,  8,  4,  6, 15, 11,  1, 10,  9,  3, 14,  5,  0, 12,  7],
              [ 1, 15, 13,  8, 10,  3,  7,  4, 12,  5,  6, 11,  0, 14,  9,  2],
              [ 7, 11,  4,  1,  9, 12, 14,  2,  0,  6, 10, 13, 15,  3,  5,  8],
              [ 2,  1, 14,  7,  4, 10,  8, 13, 15, 12,  9,  0,  3,  5,  6, 11]]
            ]

        bit6=''
        SB=[]
        n=0
        for i in range(8):
            bit6 = bit48_6b[i]
            ligne = int(bit6[0] + bit6[5] ,2) #la ligne est donnes par le premier et le dernier bit des 6bits
            colonne = int (bit6[1:5],2)       #la colonne est donnespar  Les 4 bits du milieu des 6bits
            n=s[i][ligne][colonne]
            #print(f"n {n}")
            n=bin(n)[2:].zfill(4)

            SB.append(n)
        SB = ''.join(SB)
        #print(f"Sb {SB}")
        return SB

    def permute(sb):
        # La dernière étape du calcul de f consiste à effectuer une permutation P de la sortie de la S- box pour obtenir la valeur finale de f 
        p = [16,  7, 20, 21,
             29, 12, 28, 17,
              1, 15, 23, 26,
              5, 18, 31, 10,
              2,  8, 24, 14,
             32, 27,  3,  9,
             19, 13, 30,  6,
             22, 11,  4, 25]
        #permutation de sb suivant p
        p_sb=''
        for i in p :
            p_sb += sb[i-1]
        #print(f"permuation de sb {p_sb}")
        return p_sb
    
    dict_keysl={}
    dict_keysl[0] ,dict_keysl[1] = lo ,ro # l1 = Ro
    dict_keysr={}
    dict_keysr[0] = ro
    for i in range(1,17):
        sxor=''
        e = extension( dict_keysr[i-1])
        x = xor( e, k[i-1] )
        b6 =  bit_par_6( x )
        s = Sbox(b6)
        p = permute(s)
        f = p
        t = dict_keysl[i-1] 
        for j in range(32):
            if f[j]==t[j]:
                sxor += '0'
            else: 
                sxor += '1'              
        dict_keysl[i]=dict_keysr[i-1]
        dict_keysr[i]=sxor
       
    #Nous inversons ensuite l'ordre des deux blocs dans le bloc 64 bits R 16 L 16 
    #et appliquer une permutation finale IP -1 telle que définie par le tableau suivant:
    ip_1 =  [40 ,8 ,48 ,16 ,56 ,24 ,64 ,32
            ,39 ,7 ,47 ,15 ,55 ,23 ,63 ,31
            ,38 ,6 ,46 ,14 ,54 ,22 ,62 ,30
            ,37, 5, 45 ,13 ,53 ,21 ,61 ,29
            ,36 ,4 ,44 ,12 ,52 ,20 ,60 ,28
            ,35 ,3 ,43 ,11 ,51 ,19 ,59 ,27
            ,34 ,2 ,42 ,10 ,50 ,18 ,58 ,26
            ,33 ,1 ,41 ,9 ,49 ,17 ,57 ,25
            ]
    RL16 = dict_keysr[16] + dict_keysl[16]
    cipher=''
    for i in ip_1:
        cipher += RL16[i-1] 
    
    return cipher
    

determination de Cn et Dn avec 1<=n<=16


In [77]:
M = '0000000100100011010001010110011110001001101010111100110111101111'
cipher_system_des(M )

Permutation de  0000000100100011010001010110011110001001101010111100110111101111 suivant ip : 
 k_ip = 1100110000000000110011001111111111110000101010101111000010101010
determination de Ln et Rn avec 1<=n<=16


'1000010111101000000100110101010000001111000010101011010000000101'

### 3_Decipher_DES

In [ ]:
def decipher_system_des(cipher,k):
    return decipher

# AES

### 1_Genkey_AES

In [ ]:
def clefs genkey_system_aes(n):
    return key 

In [ ]:
def genkey_subkeys_system_aes(k):
    return 

### 2_Cipher_AES

In [ ]:
def cipher_system_aes(m,k):
    return cipher

### 3_Decipher_AES

In [ ]:
def decipher_system_aes(cipher,k):
    return decipher

# Partie 3: Modes d’opérations et DES, AES